In [1]:
import pandas as pd

In [3]:
from gensim import models

In [4]:
import jieba

In [2]:
df = pd.read_csv('../data/raw/solr-chinese-domain_only-04-23-2017-cleaned.csv')

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [417]:
df['domain'].unique()

array(['qulishi.com', 'weilishi.org', 'y5000.com', 'zggdwx.com', 'artx.cn',
       'ctext.org', 'fudan.edu.cn', 'gs5000.cn', 'guoxue.com', 'hxlsw.com',
       'xuehistory.com', 'chiculture.net'], dtype=object)

In [425]:
test = df[df['domain'] != 'qulishi.com']

In [426]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143473 entries, 250 to 264831
Data columns (total 11 columns):
id           143473 non-null object
content      143473 non-null object
title        143473 non-null object
_version_    143473 non-null int64
cache        260 non-null object
segment      143473 non-null int64
digest       143473 non-null object
boost        143473 non-null float64
tstamp       143473 non-null object
url          143473 non-null object
domain       143473 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 13.1+ MB


In [422]:
msk = test['content'].str.contains('T | T')

In [379]:
msk = test[df['content'].str.split('T | T').str.len() > 1]

/Library/Python/2.7/site-packages/ipykernel-4.5.2-py2.7.egg/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [421]:
msk.info()

AttributeError: 'Series' object has no attribute 'info'

In [423]:
check = test[msk]

In [424]:
check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30306 entries, 353 to 264793
Data columns (total 11 columns):
id           30306 non-null object
content      30306 non-null object
title        30306 non-null object
_version_    30306 non-null int64
cache        103 non-null object
segment      30306 non-null int64
digest       30306 non-null object
boost        30306 non-null float64
tstamp       30306 non-null object
url          30306 non-null object
domain       30306 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 2.8+ MB


In [409]:
def extract_content(x):
    ret =  x.split('T | T')
    if (len(ret) > 1):
        r = ret[1].decode('utf-8').split(u'分页：')
        return r[0]
    else:
        return None

In [415]:
test['content_s'] = test.content.map(lambda x: extract_content(x))

/Library/Python/2.7/site-packages/ipykernel-4.5.2-py2.7.egg/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [416]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143473 entries, 250 to 264831
Data columns (total 12 columns):
id           143473 non-null object
content      143473 non-null object
title        143473 non-null object
_version_    143473 non-null int64
cache        260 non-null object
segment      143473 non-null int64
digest       143473 non-null object
boost        143473 non-null float64
tstamp       143473 non-null object
url          143473 non-null object
domain       143473 non-null object
content_s    0 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 14.2+ MB


In [405]:
idx = 0
for i in range(100, len(test)):
    t = test.iloc[i]['content_s']
    if t == None and idx < 100:
        print i
        print test.iloc[i]['url']
        print '------------------'
        idx += 1

KeyError: 'content_s'

In [402]:
print test.iloc[986]['content']

7月小微企业运行​指数​为45.6 较上月下降0.2个点 首页 明星百科 最新八卦 电影电视 音乐演出 最新综艺 独家猛料 图片写真 明星周边 7月小微企业运行​指数​为45.6 较上月下降0.2个点 yozoh 2016年09月21日 08:23:54 “经济日报—中国邮政储蓄银行小微企业运行指数”报告 经济日报—中国邮政储蓄银行小微企业运行指数课题组 (2016年7月) 2016年7月份，“经济日报-中国邮政储蓄银行小微企业运行指数”(见图1)为45.6，较上月下降0.2个点，降幅较上月扩大0.1个点，表明本月小微企业总体运行受阻，经济增长微观活力不足。 从各分项指标指数(见图2)来看，呈现“一升七降”态势。其中市场指数为42.9，较上月下降0.1个点；采购指数为44.4，较上月下降0.4个点；绩效指数为44.9，下降0.1个点；扩张指数为44.6，下降0.4个点；信心指数为47.2，下降0.1个点；融资指数为49.4，下降0.3个点；风险指数为50.9，下降0.2个点；成本指数为63.8，上升0.6个点。 从六大区域指数(见图3)来看，呈现“全部下降”态势。华北地区小微指数为44.7，下降0.1个点；东北地区为43.8，下降0.2个点；华东地区为46.8，下降0.3个点；中南地区为48.4，下降0.3个点；西南地区为46.3，下降0.1个点；西北地区为43.0，下降0.4个点。 一、按行业划分 7月七大行业小微指数呈现“两升五降”态势，小微企业总体运行受阻，经济增长微观活力不足。 从小微企业各行业运行指数(见图4)来看，除住宿餐饮业和服务业外，其他行业小微企业指数均出现下滑。 7月份农林牧渔业小微企业运行指数为44.5，较上月下降0.4个点。其市场指数为41.9，较上月下降0.3个点；采购指数为44.3，较上月下降0.5个点；绩效指数为42.8，下降0.4个点。调研结果显示，7月农林牧渔业小微企业订单量下降1.1个点，主营业务收入下降0.6个点，原材料采购量下降0.8个点，利润下降0.4个点，毛利率下降0.6个点。 上一篇 1 2 3 4 5 下一页 末页 下一篇 热门图片 在古代参与选美的都是 动作巨星李小龙死因 慈禧老佛爷的特殊爱好 同治帝惨死于梅毒病下 猜你喜欢 那些令妓女心惊肉跳的 刘备的私生活换妻如换 关羽是被诸葛亮害死的 朱元璋死后有多少美女 回顾

In [392]:
rst = test[test['content_s'] == None ]

In [394]:
rst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
id           0 non-null object
content      0 non-null object
title        0 non-null object
_version_    0 non-null int64
cache        0 non-null object
segment      0 non-null int64
digest       0 non-null object
boost        0 non-null float64
tstamp       0 non-null object
url          0 non-null object
domain       0 non-null object
content_s    0 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 0.0+ bytes


In [393]:
rst.iloc[0]['content']

IndexError: single positional indexer is out-of-bounds

In [382]:
for i in range(0, len(msk)):
    print i
    #print test.iloc[i]['url']
    extract_content(msk.iloc[i]['content'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: list index out of range

In [366]:
tt = test.iloc[917]['content']

In [368]:
if 'T | T' in tt:
    print False

In [353]:
ret = tt.split('T | T')

In [354]:
len(ret)

1

In [356]:
print tt

爱新觉罗·溥仪生平简介_爱新觉罗·溥仪的妻子_爱新觉罗·溥仪被废后的生活_爱新觉罗·溥仪的死因-趣历史 人物 - 专题 - 影视 - 社会 - 解梦 - 百家姓 - 成语 - 美女 - 水浒 - 明星 - 民族 - 星座 - 古玩 - 养生 - 娱乐 - 手机版 首页 风云人物 历史解密 战史风云 野史秘闻 文史百科 国学文化 老照片 诗词名句 成语典故 神话故事 传统文化 古文名著 热门人物: 华妃 甄宓 姜子牙 秦始皇 朱元璋 袁天罡 宋太祖 热门专题: 丝绸之路 玄武门之变 北周皇帝列表 百团大战 康熙皇帝 北魏皇帝列表 当前位置： 首页 > 爱新觉罗·溥仪 > 资讯列表 分页： 1/1 页 首页 上一页 1 下一页 尾页 末代皇帝爱新觉罗·溥仪出生 2017-02-07 历史对溥仪的评价 爱新觉罗溥仪简介 2016-10-27 爱新觉罗·慧生是谁？慧生与溥仪是什么关系 2016-09-25 一组清朝末年的“爱新觉罗”宗室人物的旧照 2016-09-17 一组末代皇帝爱新觉罗·溥仪的绝密私房照片 2016-09-13 爱新觉罗溥伟：三次错过皇位 心比天高却命比纸薄 2016-04-14 清朝爱新觉罗皇族绝后的原因竟是乱伦婚配？ 2016-03-31 溥仪叔父载涛：从王爷到人民解放军马政局顾问 2016-01-16 溥仪有子女吗？爱新觉罗·溥仪的后人今何在 2015-11-23 爱新觉罗·溥伟生平简介及怎么死的？ 2015-10-13 爱新觉罗·溥任简介 末代皇帝溥仪的弟弟溥任生平 2015-04-15 分页： 1/1 页 首页 上一页 1 下一页 尾页 相关人物 李玉琴 武乙 廪辛 祖甲 最新更新 人物 解密 战史 野史 文史 文化 1 历史上哪位君王颁发的免死令最不靠谱？ 2 此人是康熙大帝的重要谋臣 最后却发配边疆 3 历史上的关羽有多厉害 这场战斗揭晓答案 4 爱哭的刘皇叔：他的用人诀窍是什么？ 5 宋太宗好用老成官员 寇准被迫吃药扮老大爷 1 中国历史上让汉人引以为耻的三件事分别是什么 2 三峡悬棺是怎么放上去的?揭秘三种摆放方法 3 千年女尸出土时竟然说话了?揭秘四大谜团 4 清朝最想把大贪官和珅拉下马的官员是谁？ 5 明朝重臣解缙：为何被锦衣卫埋入雪中冻死？ 1 巴拉圭战争背景与经过 巴拉圭战争结果与影响 2 耶拿战役的时间 耶拿战役有什么影响 3

In [331]:
print test.iloc[917]['url']

http://www.qulishi.com/renwu/aixinjueluopuyi/news.html


In [314]:
df['domain'].value_counts()

qulishi.com       121359
artx.cn            79051
gs5000.cn          17896
weilishi.org       14807
y5000.com          11914
hxlsw.com           7140
zggdwx.com          6502
guoxue.com          5416
fudan.edu.cn         408
ctext.org            261
xuehistory.com        77
chiculture.net         1
Name: domain, dtype: int64

In [63]:
global st_more
st_more = [u'：', u'去']

In [13]:
def load_stopwords(path='../data/stop_words/chinese_stops.txt'):  
    """ 
    加载停用词 
    :param path: 
    :return: 
    """  
    with open(path) as f:  
        stopwords = filter(lambda x: x, map(lambda x: x.strip().decode('utf-8'), f.readlines()))  
    stopwords.extend([' ', '\t', '\n'])  
    return frozenset(stopwords)  

In [11]:
def cut_words(sentence):  
    """ 
    分词 
    :param sentence: 
    :return: 
    """  
    stopwords = load_stopwords()  
    return filter(lambda x: not stopwords.__contains__(x), jieba.cut(sentence))  

In [21]:
import operator
 
def isAscii(string):
    return reduce(operator.and_, 
    [ord(x) < 256 for x in string],True)

In [239]:
def get_title_sent(x):
    s = x.decode('utf-8')
    title_seg = cut_words(s)
    s1 = [ t for t in title_seg if t not in st_more]
    s2 = models.doc2vec.LabeledSentence(words=s1, tags=['TITLE'])
    return s2

In [280]:
print len(df.iloc[1]['content'].split('\n'))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 0: ordinal not in range(128)

In [278]:
df.iloc[1]['url']

'http://www.qulishi.com/news/201701/157540.html'

In [286]:
for t in df.iloc[1]['content'].decode('utf-8').split(u' '):
    print t, len(t)
    print '******************'

文宣宗高洋：怀揣着爱妃的头颅去赴宴 17
******************
趣历史首页 5
******************
|  讲述历史上那些有趣的事 14
******************
人物 2
******************
- 1
******************
专题 2
******************
- 1
******************
影视 2
******************
- 1
******************
社会 2
******************
- 1
******************
解梦 2
******************
- 1
******************
百家姓 3
******************
- 1
******************
成语 2
******************
- 1
******************
美女 2
******************
- 1
******************
水浒 2
******************
- 1
******************
明星 2
******************
- 1
******************
民族 2
******************
- 1
******************
星座 2
******************
- 1
******************
古玩 2
******************
- 1
******************
养生 2
******************
- 1
******************
娱乐 2
******************
- 1
******************
手机版 3
******************
首页 2
******************
野史秘闻 4
******************
演义趣闻 4
******************
> 1
******************
正文 2
******************
文宣宗高洋：怀揣着爱妃的头颅去赴宴 17
******************
20

In [300]:
import re
def get_content_sent(x):
    global st_more
    v = list()
    idx = 0
    
    s = x.decode('utf-8')
    #sentences = re.split(r'\s{2,}', x)
    sentences = s.split(u' ')
    print len(sentences)
    for t in sentences:
        if len(t) > 20:
            print t, len(t)
            s2 = cut_words(t)
            ret = [ r for r in s1 if r not in st_more]
            sentence = models.doc2vec.LabeledSentence(words=s2, tags=['SENT_%s' % (idx+1)])
            v.append(sentence)
            idx += 1
    return v

In [253]:
def get_n_sentence(x, k):
    s = x.decode('utf-8')
    sentences = s.split(u'。')
    print len(sentences)
    return sentences[k]

In [307]:
ret = get_content_sent(df.iloc[7]['content'])

451
春秋》也有类似的记载，“(操)姿貌短小”。 21
志》)，也就是说，刘备的老板是他妈，他妈是个摆地摊的。 27
　　中国女性们向来有颗驿动的心，数千年来，万千粉黛们的择偶观一直飘忽不定，无论是寒酸的文艺青年 47
，都曾在女人们的情爱史上，泼下了隆重的笔墨。抛开悠远的历史，拿近点说，自建国以来，女同志们的审美标准也由 52
的“工农兵”，悄然渐变成如今的“高富帅”。不过，倘若把如今女子的眼光移步到英雄辈出的三国时代，我们会惊奇地发现，竟然鲜有男人能通过“考核”。即使三国时代的缔造者曹操 82
　　当然，我们可以先拿三国的“男一号”曹操同志做个参照。曹操是个“富二代”，这是没有任何非议的。曹操的老爹 53
末年的朝廷，有个离奇的制度，就是花钱能买官。曹嵩某日 26
，就一掷千金给自己捐了太尉一职，过了一把官瘾。如此殷实的家境，曹操做人的底气，自然“宏大”了不少。不过，人生最悲催的事情恰恰在于，再多的钞票，也叠不出三公分的高度，当然，这事居然让曹操给遇到了。 97
揭秘：曹操为延续中国文脉做了哪些强悍贡献？ 21
三国陈宫为什么能说服曹操善待厚养自己的家人 21
历史上十大隐蔽很好的伪君子：刘备诸葛亮上榜 21
大唐帝国为何被称为“脏唐”？只因帝王带头乱伦 22
清朝唐朝文化：中国历史上第一个状元诞生于唐朝 22
夏朝之前的历史是什么？夏朝之前还有朝代吗？ 21
曾国藩创立的湘军从太平天国一直打到解放战争 21
　　1851年，清咸丰元年，太平天国起义爆发，这场起于广西的农民起义如野火般迅速 40
吴三桂是明末出色的军事家，屈服了清政府，为清政府打下了半个江 30
两位最伟大的皇帝但历史记忆却差距如此之大？ 21
　　中国历史文明璀璨，各大王朝都留有一份属于自己的历史记忆。那今天我们提到的两位 40
　　在历史上，刘禅一直被当作烂泥扶不上壁!原因无它，小的时候被愤怒刘备砸在地上， 40
　　正德十六年(1521年)四月二十日，明武宗驾崩于豹房。由于明武宗膝下无子，也 40
　　　历史杂货铺解读历史第三十一讲，独家原创，如需转载，请注明出处!导语：汉武帝 40
清朝的十二位皇帝中，最有玩味价值的就是清高宗乾隆了。康熙和雍正爷俩辛辛 35
　　梁山都知道有一百零八将，可是这个顺序不是随便谁来的早就是谁的，而是有着其中的 40
　　《云别传》说他长得高大挺拔、仪表堂堂

In [308]:
ret.append(get_title_sent(df.iloc[8]['title']))

In [309]:
mm = models.Doc2Vec(ret, size=len(ret), min_count=1)

In [310]:
mm.docvecs.most_similar('TITLE', topn=5)

[('SENT_47', 0.361654669046402),
 ('SENT_70', 0.3287051320075989),
 ('SENT_72', 0.19312149286270142),
 ('SENT_69', 0.17297092080116272),
 ('SENT_60', 0.1701260656118393)]

In [324]:
ret =  df.iloc[1]['content'].split('T | T')
r = ret[1].decode('utf-8').split(u'分页：')
print r[0]

 　　文宣宗 高洋 是 北齐 的开国 皇帝 ，他一生传奇，他曾提着爱妃的头颅赴宴并当场剖解尸体，这是为何呢? 　　清河王高岳的宴会上，两个陪酒女郎很是惹眼。只见她们衣着暴露，笑脸旖旎，两双狐媚眼更是勾魂摄魄，惹得众多参加宴会的男人 垂涎三尺 ，纷纷打听哪里来的两个尤物。知情者说，她二人 姓薛 ，是姐妹，本地新崛起的倡界 明星 ，要貌有貌，要才有才，吹拉弹唱无所不能。 网络配图 　　“高岳这家伙真有本事。”一个人喝了口酒，掩饰住差点流出来的哈喇子，微醉的眼神迷离地望着 薛氏 姐妹。 　　“据说，高岳跟薛家姐妹……” 　　说话人硬生生咽下后半句，神情也变得严肃起来。因为他发现，文宣帝高洋正色迷迷地望着薛氏姐妹。皇上看上的女人，他们可不敢再胡乱非议。 


In [312]:
print df.iloc[8]['title']

揭开古埃及法老王图坦卡蒙死亡真相之谜


In [313]:
df.iloc[8]['url']

'http://www.qulishi.com/news/201701/160186.html'

In [299]:
print df.iloc[1]['content']

文宣宗高洋：怀揣着爱妃的头颅去赴宴 趣历史首页 |  讲述历史上那些有趣的事 人物 - 专题 - 影视 - 社会 - 解梦 - 百家姓 - 成语 - 美女 - 水浒 - 明星 - 民族 - 星座 - 古玩 - 养生 - 娱乐 - 手机版 首页 野史秘闻 演义趣闻 > 正文 文宣宗高洋：怀揣着爱妃的头颅去赴宴 2017-01-13 17:08:48 来源： 趣历史 责任编辑：zhangmiao 字号： T | T 　　文宣宗 高洋 是 北齐 的开国 皇帝 ，他一生传奇，他曾提着爱妃的头颅赴宴并当场剖解尸体，这是为何呢? 　　清河王高岳的宴会上，两个陪酒女郎很是惹眼。只见她们衣着暴露，笑脸旖旎，两双狐媚眼更是勾魂摄魄，惹得众多参加宴会的男人 垂涎三尺 ，纷纷打听哪里来的两个尤物。知情者说，她二人 姓薛 ，是姐妹，本地新崛起的倡界 明星 ，要貌有貌，要才有才，吹拉弹唱无所不能。 网络配图 　　“高岳这家伙真有本事。”一个人喝了口酒，掩饰住差点流出来的哈喇子，微醉的眼神迷离地望着 薛氏 姐妹。 　　“据说，高岳跟薛家姐妹……” 　　说话人硬生生咽下后半句，神情也变得严肃起来。因为他发现，文宣帝高洋正色迷迷地望着薛氏姐妹。皇上看上的女人，他们可不敢再胡乱非议。 分页：1/3页   上一页 1 2 3 下一页 相关阅读推荐： 历史热图 清代街头小贩照如时尚大片 实拍美女纹身被占便宜 旧时期朝鲜人民的生活状态 上个世纪越南人的生活场景 坦桑尼亚人民状态的照片 18世纪时乌克兰人的老照片 “梦幻客轮”泰坦尼克号的旧影 20世纪初期俄罗斯人民状态 更多 看过本文的人还看过 · 文宣宗高洋：怀揣着爱妃的头颅去赴宴 · 文宣宗高洋：怀揣着爱妃的头颅去赴宴 热门推荐 风云人物 历史解密 战史风云 野史秘闻 文史百科 清朝历史上唯一被老婆和女儿毒死的皇帝是谁 武则天还政李唐的原因？绝非就因狄仁杰劝说 · 唐朝那些出家当道士的公主竟然有五位 · 历史上为何古代女性中只有武则天能当上皇帝 · 这皇帝竟是个守财奴！因不肯花银子丢了江山 · 关羽死亡之谜：当年是谁谋杀了关羽 · 曹操最欣赏的一位接班人曹冲是怎么死的？ 清朝十三陵谜案：三位大明皇帝为何不知所踪 揭秘乾隆裕陵地宫八尊菩萨暗藏惊人秘密 · 唐高祖李渊为何禅位给二子李世民？ · 西游中唯一让妖怪都恐惧的凡人究竟是何来历 · 清末立宪运动：一

In [260]:
print get_n_sentence(df.iloc[1]['content'], 7)

32
皇上看上的女人，他们可不敢再胡乱非议


In [261]:
print get_n_sentence(df.iloc[1]['content'], 29)

32
在繁荣的朝代诞生了许多杰出的 详情 苏轼为什么叫苏东坡 苏轼开创了什么派 　　苏轼是我国宋代著名文学家，字子瞻，他精通诗词书画，作品众多，词作属于豪放一派 详情 八大山人是八个人吗 八大山人的印章的特点 　　最近几年，八大山人的书画成了书画爱好家们互相争抢的珍藏品


In [262]:
print get_n_sentence(df.iloc[1]['content'], 17)

32
熙宁九年(1076)，苏东 详情 晚清皇后通奸之谜：婉容两位情人的最后结局 　　婉容两个情人的最后结局　　发生在伪满内廷的皇后婉容通奸之事，据笔者调查，当时 详情 惊呆了！揭秘世界上唯一没有寡妇的民族 　　西藏往西，有个国家叫尼泊尔，大家应该都听过，但你不一定知道它有个古老的民族， 详情 历史上桃园结义中为何刘备是老大？ 在《三国演义》中，桃园结义是刘关张三人的历史起点，也是他们友谊的见证点


In [264]:
print df.iloc[1]['content']

文宣宗高洋：怀揣着爱妃的头颅去赴宴 趣历史首页 |  讲述历史上那些有趣的事 人物 - 专题 - 影视 - 社会 - 解梦 - 百家姓 - 成语 - 美女 - 水浒 - 明星 - 民族 - 星座 - 古玩 - 养生 - 娱乐 - 手机版 首页 野史秘闻 演义趣闻 > 正文 文宣宗高洋：怀揣着爱妃的头颅去赴宴 2017-01-13 17:08:48 来源： 趣历史 责任编辑：zhangmiao 字号： T | T 　　文宣宗 高洋 是 北齐 的开国 皇帝 ，他一生传奇，他曾提着爱妃的头颅赴宴并当场剖解尸体，这是为何呢? 　　清河王高岳的宴会上，两个陪酒女郎很是惹眼。只见她们衣着暴露，笑脸旖旎，两双狐媚眼更是勾魂摄魄，惹得众多参加宴会的男人 垂涎三尺 ，纷纷打听哪里来的两个尤物。知情者说，她二人 姓薛 ，是姐妹，本地新崛起的倡界 明星 ，要貌有貌，要才有才，吹拉弹唱无所不能。 网络配图 　　“高岳这家伙真有本事。”一个人喝了口酒，掩饰住差点流出来的哈喇子，微醉的眼神迷离地望着 薛氏 姐妹。 　　“据说，高岳跟薛家姐妹……” 　　说话人硬生生咽下后半句，神情也变得严肃起来。因为他发现，文宣帝高洋正色迷迷地望着薛氏姐妹。皇上看上的女人，他们可不敢再胡乱非议。 分页：1/3页   上一页 1 2 3 下一页 相关阅读推荐： 历史热图 清代街头小贩照如时尚大片 实拍美女纹身被占便宜 旧时期朝鲜人民的生活状态 上个世纪越南人的生活场景 坦桑尼亚人民状态的照片 18世纪时乌克兰人的老照片 “梦幻客轮”泰坦尼克号的旧影 20世纪初期俄罗斯人民状态 更多 看过本文的人还看过 · 文宣宗高洋：怀揣着爱妃的头颅去赴宴 · 文宣宗高洋：怀揣着爱妃的头颅去赴宴 热门推荐 风云人物 历史解密 战史风云 野史秘闻 文史百科 清朝历史上唯一被老婆和女儿毒死的皇帝是谁 武则天还政李唐的原因？绝非就因狄仁杰劝说 · 唐朝那些出家当道士的公主竟然有五位 · 历史上为何古代女性中只有武则天能当上皇帝 · 这皇帝竟是个守财奴！因不肯花银子丢了江山 · 关羽死亡之谜：当年是谁谋杀了关羽 · 曹操最欣赏的一位接班人曹冲是怎么死的？ 清朝十三陵谜案：三位大明皇帝为何不知所踪 揭秘乾隆裕陵地宫八尊菩萨暗藏惊人秘密 · 唐高祖李渊为何禅位给二子李世民？ · 西游中唯一让妖怪都恐惧的凡人究竟是何来历 · 清末立宪运动：一

In [49]:
i = 0
title_seg = cut_words(df.iloc[1]['title'])

In [153]:
s1 = [ t for t in title_seg if t not in st_more]

In [65]:
t = ', '.join(ret)
print t

文宣宗, 高洋, 怀揣, 爱妃, 头颅, 赴宴


In [47]:
s = load_stopwords()  

In [108]:
s = df.iloc[i]['content'].decode('utf-8')

In [109]:
len(s)

6778

In [115]:
sentences = s.split(u'。')

In [116]:
len(sentences)

43

In [ ]:
s_seg = []

In [230]:
model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)

In [231]:
title_seg = cut_words(df.iloc[i]['title'])
s1 = [ t for t in title_seg if t not in st_more]
s2 = models.doc2vec.LabeledSentence(
words=ret, tags=['SENT_0'])

In [232]:
s = df.iloc[i]['content'].decode('utf-8')
sentences = s.split(u'。')

In [233]:
v = list()
v.append(s2)
idx = 0
for t in sentences:
    s = cut_words(t)
    ret = [ r for r in s if r not in st_more]
    sentence = models.doc2vec.LabeledSentence(
    words=ret, tags=['SENT_%s' % (idx+1)])
    v.append(sentence)
    idx += 1
    #print ', '.join(ret)
    #print '----------------'

In [213]:
model.build_vocab(v)

In [214]:
len(v)

44

In [215]:
for epoch in range(10):
    model.train(v)
    model.alpha -= 0.002  # decrease the learning rate`
    model.min_alpha = model.alpha  # fix the learning rate, no decay

ss = sentences.append(s1)

In [234]:
m1 = models.Doc2Vec(v, size=44, min_count=1)

In [237]:
m1.docvecs.most_similar('SENT_0', topn=5)

[('SENT_15', 0.9279123544692993),
 ('SENT_1', 0.8824229836463928),
 ('SENT_25', 0.8671115040779114),
 ('SENT_43', 0.8581242561340332),
 ('SENT_21', 0.8455114364624023)]

In [229]:
print sentences[42]

欧阳询最为后世人称道的就是 详情 热门 图文 被老婆女儿毒死的皇帝 揭秘不同年代洞房之夜 三位大明皇帝不知所踪 武则天还政李唐的原因 大唐为何被称“脏唐” 裕陵地宫八尊菩萨秘密 唐朝当道士公主有五位 大清唯一被凌迟的公主 最新 排行 点击排行 图库排行 专题排行 1 惊悚刑具：让古代女囚生不如死的酷刑“铁莲花” 2 《山海经》中的上古神兽们！青龙鲛人大集合 3 揭秘历史上真实存在的10大酷刑：令人生不如死！ 4 侵华日军虐待中国女俘惊人手段！用刑惨绝人寰 5 中国历史上死得最窝囊的九大战神：英雄无善终！ 6 中国史上被当众施裸刑的六位女人：女子裸刑秘闻 7 “杂牌将军”赵云为什么会成为最受欢迎三国武将？ 8 真正的武林高手！盘点近代史上的32位国术大师 9
